In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from tqdm import tqdm
import urllib.parse
import pandas as pd
import re

In [ ]:
def get_long_lat(location, kabupaten):
    import time
    # Gabungkan location dan kabupaten untuk query lebih presisi
    search_query = f"{location} {kabupaten}"
    encoded_location = urllib.parse.quote(search_query)

    driver.get("https://www.google.com/maps/search/{}".format(encoded_location))
    driver.implicitly_wait(3)
    
    # Get the initial page source
    initial_page_source = driver.page_source
    
    # Wait for the page source to change with a timeout of 10 seconds
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.page_source != initial_page_source)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # CASE 1: Ditemukan list banyak hasil pencarian
    if soup.find('h1', class_="fontTitleLarge IFMGgb"):
        print("Case 1: Multiple results ditemukan")
        
        # Ambil link dari hasil pencarian pertama
        link_element = soup.find('a', class_='hfpxzc')
        if link_element:
            url = link_element.get('href')
            print(f"Link: {url}")
            driver.get(url)  # Buka link
            
            # Tunggu page source berubah setelah klik
            initial_page_source = driver.page_source
            wait = WebDriverWait(driver, 10)
            wait.until(lambda driver: driver.page_source != initial_page_source)
            
            # Tunggu URL berubah menjadi format yang stabil (dengan /@lat,lng)
            for attempt in range(10):
                current_url = driver.current_url
                if '/@' in current_url and re.search(r'/@-?\d+\.\d+,-?\d+\.\d+', current_url):
                    print(f"URL stabil ditemukan pada attempt {attempt}")
                    break
                if attempt < 9:
                    time.sleep(1)
            
            # Ambil soup dari halaman setelah klik
            html_after_click = driver.page_source
            soup_after_click = BeautifulSoup(html_after_click, "html.parser")
            
            # Ambil nama Google dari h1 dengan class "DUwDvf lfPIob"
            h1_element = soup_after_click.find('h1', class_='DUwDvf lfPIob')
            if h1_element:
                nama_google = h1_element.get_text(strip=True)
                print(f"Nama Google: {nama_google}")
            else:
                nama_google = None
                print("Nama Google tidak ditemukan")
            
            # Extract koordinat dari URL
            current_url = driver.current_url
            print(f"Final URL: {current_url}")
            
            lat_lng_match = re.search(r'/@(-?\d+\.\d+),(-?\d+\.\d+)', current_url)
            if lat_lng_match:
                latitude = float(lat_lng_match.group(1))
                longitude = float(lat_lng_match.group(2))
                print(f"Latitude: {latitude}, Longitude: {longitude}")
                return [latitude, longitude, nama_google, current_url, 0]
            else:
                print("Koordinat tidak ditemukan di URL")
                return [None, None, nama_google, current_url, 1]
        else:
            print("Link tidak ditemukan")
            return [None, None, None, None, 1]
    
    # CASE 2: Ditemukan tepat 1 hasil pencarian langsung
    elif soup.find('h1', class_='DUwDvf lfPIob'):
        print("Case 2: 1 hasil ditemukan langsung")
        
        # Tunggu URL berubah menjadi format yang stabil (dengan /@lat,lng)
        for attempt in range(10):
            current_url = driver.current_url
            if '/@' in current_url and re.search(r'/@-?\d+\.\d+,-?\d+\.\d+', current_url):
                print(f"URL stabil ditemukan pada attempt {attempt}")
                break
            if attempt < 9:
                time.sleep(1)
        
        # Ambil nama Google
        h1_element = soup.find('h1', class_='DUwDvf lfPIob')
        if h1_element:
            nama_google = h1_element.get_text(strip=True)
            print(f"Nama Google: {nama_google}")
        else:
            nama_google = None
            print("Nama Google tidak ditemukan")
        
        # Extract koordinat dari URL
        current_url = driver.current_url
        print(f"Final URL: {current_url}")
        
        lat_lng_match = re.search(r'/@(-?\d+\.\d+),(-?\d+\.\d+)', current_url)
        if lat_lng_match:
            latitude = float(lat_lng_match.group(1))
            longitude = float(lat_lng_match.group(2))
            print(f"Latitude: {latitude}, Longitude: {longitude}")
            return [latitude, longitude, nama_google, current_url, 0]
        else:
            print("Koordinat tidak ditemukan di URL")
            return [None, None, nama_google, current_url, 1]
    
    # CASE 3: Tidak ditemukan
    else:
        print("Case 3: Lokasi tidak ditemukan")
        return [None, None, None, None, 2]

In [ ]:
file_input = 'Flat_sekolah_terdampak.csv'

df = pd.read_csv('input/{}'.format(file_input), sep=';')

if 'latitude' not in df.columns:
    df['latitude'] = ''

if 'longitude' not in df.columns:
    df['longitude'] = ''

if 'nama_google' not in df.columns:
    df['nama_google'] = ''

if 'url' not in df.columns:
    df['url'] = ''

In [ ]:
df_na = df.loc[df['latitude'].isna() | (df['latitude'] == '')]
df_no_na = df.loc[~(df['latitude'].isna() | (df['latitude'] == ''))]

# Untuk df_na: set isna=1 kecuali sudah 2 (tidak ditemukan sebelumnya)
df_na['isna'] = df_na['isna'].apply(lambda x: 1 if (pd.isna(x) or x != 2) else 2)
# Untuk df_no_na: set isna=0 (ditemukan)
df_no_na['isna'] = 0

C:\Users\fawcet.makay\AppData\Local\Temp\ipykernel_23796\1227467311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_na['isna'] = df_na['isna'].apply(lambda x: 1 if (pd.isna(x) or x != 2) else 2)
C:\Users\fawcet.makay\AppData\Local\Temp\ipykernel_23796\1227467311.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_na['isna'] = 0


In [ ]:
df_na.head(5)

,npsn_isian,nama_isian,Jenjang,alamat_jalan,nama_dusun,desa_kelurahan,kecamatan,kabupaten,provinsi,lintang,bujur,latitude,longitude,nama_google,url,isna
0,69775026,AL0RIFDA,PAUD Sederajat,"PERUMNAS BRANDAN PERMAI, TANGKAHAN DURIAN",-,Tangkahan Durian,Kec. Berandan Barat,Kab. Langkat,Prov. Sumatera Utara,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2
1,69790018,IKRA,PAUD Sederajat,BATANG DURIAN NO 1,-,PEKAN TANJUNG PURA,Kec. Tanjung Pura,Kab. Langkat,Prov. Sumatera Utara,-5.443.757.000.000,114.421.127.000.000,NaN,NaN,NaN,NaN,2
4,69932645,KB AL-BARRU,PAUD Sederajat,Gampong Lhok Merbo Kecamatan Sawang Kabupaten ...,Lhok Merbo,Lhok Meureubo,Kec. Sawang,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2
7,69967119,KB AR-RIDHA,PAUD Sederajat,Gampong Cot Ara Kecamatan Baktiya Kabupaten Ac...,Cot Ara,Cot Ara,Kec. Baktiya,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2
12,69798169,KB AL-AMIN,PAUD Sederajat,Jalan Irigasi Gampong Kayee Panyang Kecamatan ...,Baroh,Kayee Panyang,Kec. Syamtalira Bayu,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)

for i, r in tqdm(df_na.iterrows(), total=df_na.shape[0]):
    if r['isna'] == 1 :
        location_name = r['nama_isian']
        kabupaten = r['kabupaten']  # Ambil kabupaten dari dataframe
        df_na.loc[i, ['latitude', 'longitude', 'nama_google', 'url', 'isna']] = get_long_lat(location_name, kabupaten)

  0%|          | 0/4649 [00:00<?, ?it/s]

Case 2: 1 hasil ditemukan langsung


  1%|▏         | 69/4649 [00:04<04:33, 16.77it/s]

URL stabil ditemukan pada attempt 3
Nama Google: Paud KB muara batubatu
Final URL: https://www.google.com/maps/place/Paud+KB+muara+batubatu/@2.6744893,97.861126,17z/data=!3m1!4b1!4m6!3m5!1s0x30300bf830a83ea7:0xb6f79d9decff084a!8m2!3d2.6744839!4d97.8637009!16s%2Fg%2F11vj999zbj?entry=ttu&g_ep=EgoyMDI2MDIwOS4wIKXMDSoASAFQAw%3D%3D
Latitude: 2.6744893, Longitude: 97.861126
Case 2: 1 hasil ditemukan langsung
URL stabil ditemukan pada attempt 3
Nama Google: Kintana Tour & Leisure
Final URL: https://www.google.com/maps/place/Kintana+Tour+%26+Leisure/@-6.2731241,106.7949156,17z/data=!4m6!3m5!1s0xa4434f221111fc39:0x3b4040f763b8f0c!8m2!3d-6.2731294!4d106.7974905!16s%2Fg%2F11q4jqwbfr?entry=ttu&g_ep=EgoyMDI2MDIwOS4wIKXMDSoASAFQAw%3D%3D
Latitude: -6.2731241, Longitude: 106.7949156
Case 2: 1 hasil ditemukan langsung


  2%|▏         | 71/4649 [00:12<16:33,  4.61it/s]

...

URL stabil ditemukan pada attempt 3
Nama Google: PAUD Siboru Tumbaga
Final URL: https://www.google.com/maps/place/PAUD+Siboru+Tumbaga/@1.9478599,98.5955287,17z/data=!3m1!4b1!4m6!3m5!1s0x302e51eaa7da8493:0x7b5fc5a0f1022e5c!8m2!3d1.9478599!4d98.5955287!16s%2Fg%2F11gjw_7x5r?entry=ttu&g_ep=EgoyMDI2MDIwOS4wIKXMDSoASAFQAw%3D%3D
Latitude: 1.9478599, Longitude: 98.5955287
Case 2: 1 hasil ditemukan langsung


 10%|▉         | 445/4649 [21:44<4:20:00,  3.71s/it]

URL stabil ditemukan pada attempt 3
Nama Google: Paud Sikarang Bungong
Final URL: https://www.google.com/maps/place/Paud+Sikarang+Bungong/@5.2533481,96.9396819,17z/data=!3m1!4b1!4m6!3m5!1s0x304771ea555b9e75:0x7c15480160e05ed4!8m2!3d5.2533481!4d96.9396819!16s%2Fg%2F11gb3tqn52?entry=ttu&g_ep=EgoyMDI2MDIwOS4wIKXMDSoASAFQAw%3D%3D
Latitude: 5.2533481, Longitude: 96.9396819
Case 2: 1 hasil ditemukan langsung


 10%|▉         | 445/4649 [21:46<3:25:45,  2.94s/it]


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=144.0.7559.133); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff64756f3d5
	0x7ff64756f430
	0x7ff6473110bd
	0x7ff6472fc642
	0x7ff6473229cb
	0x7ff64739d500
	0x7ff6473b9652
	0x7ff64735cb0c
	0x7ff64735da53
	0x7ff64784b470
	0x7ff64784586d
	0x7ff64786621a
	0x7ff64758b235
	0x7ff647593a5c
	0x7ff647578844
	0x7ff6475789f6
	0x7ff64755eb87
	0x7ffd9bc2e8d7
	0x7ffd9dd0c40c


In [34]:
df_na.head()

,npsn_isian,nama_isian,Jenjang,alamat_jalan,nama_dusun,desa_kelurahan,kecamatan,kabupaten,provinsi,lintang,bujur,latitude,longitude,nama_google,url,isna
0,69775026,AL0RIFDA,PAUD Sederajat,"PERUMNAS BRANDAN PERMAI, TANGKAHAN DURIAN",-,Tangkahan Durian,Kec. Berandan Barat,Kab. Langkat,Prov. Sumatera Utara,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2
1,69790018,IKRA,PAUD Sederajat,BATANG DURIAN NO 1,-,PEKAN TANJUNG PURA,Kec. Tanjung Pura,Kab. Langkat,Prov. Sumatera Utara,-5.443.757.000.000,114.421.127.000.000,NaN,NaN,NaN,NaN,2
4,69932645,KB AL-BARRU,PAUD Sederajat,Gampong Lhok Merbo Kecamatan Sawang Kabupaten ...,Lhok Merbo,Lhok Meureubo,Kec. Sawang,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2
7,69967119,KB AR-RIDHA,PAUD Sederajat,Gampong Cot Ara Kecamatan Baktiya Kabupaten Ac...,Cot Ara,Cot Ara,Kec. Baktiya,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2
12,69798169,KB AL-AMIN,PAUD Sederajat,Jalan Irigasi Gampong Kayee Panyang Kecamatan ...,Baroh,Kayee Panyang,Kec. Syamtalira Bayu,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,2


In [35]:
# Gabungkan data yang sudah diproses
x = pd.concat([df_na, df_no_na])
x = x.sort_index()

# Hapus kolom 'isna' sebelum menyimpan (opsional, jika ingin)
# x = x.drop('isna', axis=1)

# Simpan ke file output
output_file = file_input
x.to_csv("input/{}".format(output_file), sep=';', index=False)
print(f"Data berhasil disimpan ke input/{output_file}")
print(f"Total records: {len(x)}")
print(f"Records dengan data: {len(x[x['latitude'] != ''])}")
print(f"Records tanpa data: {len(x[x['latitude'] == ''])}")

Data berhasil disimpan ke input/Flat_sekolah_terdampak.csv
Total records: 4781
Records dengan data: 4781
Records tanpa data: 0
